# TFPT Playground v1.06 – Topological Fixed Point Theory

This notebook is a self contained, runnable playground that implements the newest version of the theory:
- Topological and geometric invariants: \(c_3=\tfrac{1}{8\pi}\), \(\varphi_0=\tfrac{1}{6\pi}+\tfrac{3}{256\pi^4}\)
- Cubic fixed point for \(\alpha\) with the 4D variational route
- Log exact \(E_8\) ladder \(\{D_n=60-2n\}\) with \(\gamma(0)=0.834\), \(\lambda=\gamma(0)/(\ln 248-\ln 60)\)
- Standard Model style blocks for masses and scales (EW, hadron, pion, PQ axion, seesaw)
- Inflation module (alpha attractor) with predictions for \((n_s, r, \alpha_s, n_t, H, V^{1/4})\)

> Tip: Run all cells once. Then play with the widgets near the bottom (requires ipywidgets).


In [ ]:
# Core imports
import math, numpy as np, pandas as pd
import matplotlib.pyplot as plt
from math import log, pi, sqrt
from dataclasses import dataclass
try:
    import mpmath as mp
    mp.mp.dps = 50
except Exception as e:
    raise RuntimeError("mpmath is required for high precision")

## Invariants and handy helpers

We collect all fixed numbers and helper functions in one place.

In [ ]:
# Fixed invariants (reduced Planck units unless noted otherwise)
def c3():
    return 1.0/(8.0*math.pi)

def phi0():
    return 1.0/(6.0*math.pi) + 3.0/(256.0*math.pi**4)

def A_coeff():
    # A = 2 c3^3 = 1/(256 pi^3)
    c = c3()
    return 2.0*(c**3)

def kappa():
    # kappa = (b1/(2pi)) * ln(1/phi0), with b1 = 41/10 in GUT norm
    b1 = 41.0/10.0
    return (b1/(2.0*math.pi))*math.log(1.0/phi0())

def theory_numbers():
    return {
        "c3": c3(),
        "phi0": phi0(),
        "A": A_coeff(),
        "kappa": kappa(),
        "b1": 41.0/10.0,
    }

theory_numbers()

## Cubic fixed point for \(\alpha\)

We solve the cubic
\[\alpha^3 - 2c_3^3\alpha^2 - 8\,b_1\,c_3^6 \ln(1/\varphi_0) = 0\]
with both a precise Cardano style expression and a very accurate practice formula.


In [ ]:
def alpha_root_high_precision():
    c = mp.mpf(c3())
    b1 = mp.mpf(41)/mp.mpf(10)
    phi = mp.mpf(phi0())
    # coefficients for alpha^3 - 2 c3^3 alpha^2 - 8 b1 c3^6 ln(1/phi0) = 0
    A2 = -2*(c**3)              # coefficient in front of alpha^2
    B = -8*b1*(c**6)*mp.log(1.0/phi)

    # Depressed cubic via alpha = y + 2/3 c3^3
    shift = (2*(c**3))/3
    p = - (4/3)*(c**6)
    q = - (16/27)*(c**9) + B
    # Discriminant
    Delta = (q/2)**2 + (p/3)**3
    root = mp.cbrt(-q/2 + mp.sqrt(Delta)) + mp.cbrt(-q/2 - mp.sqrt(Delta)) + shift
    return root

def alpha_root_practice():
    # alpha ≈ (8 b1 c3^6 ln(1/phi0))^(1/3) + 2/3 c3^3
    b1 = 41.0/10.0
    c = c3()
    term = 8.0*b1*(c**6)*math.log(1.0/phi0())
    return term**(1.0/3.0) + (2.0/3.0)*(c**3)

alpha_exact = float(alpha_root_high_precision())
alpha_practice = alpha_root_practice()
alpha_exact, alpha_practice, 1.0/alpha_exact

## Log exact \(E_8\) ladder

We implement \(D_n=60-2n\), \(\gamma(0)=0.834\), \(\lambda=\gamma(0)/(\ln 248 - \ln 60)\) and
\[\varphi_n = \begin{cases}
\varphi_0, & n=0,\\[2pt]
\varphi_0 \, e^{-\gamma(0)} \left(\dfrac{D_n}{D_1}\right)^\lambda, & n\ge 1.
\end{cases}\]

In [ ]:
def gamma0():
    return 0.834

def lambda_from_gamma0():
    s_star = math.log(248.0) - math.log(60.0)
    return gamma0()/s_star

def D(n):
    return 60 - 2*n

def phi_n(n):
    if n == 0:
        return phi0()
    lam = lambda_from_gamma0()
    return phi0()*math.exp(-gamma0())*((D(n)/58.0)**lam)

def ladder_table():
    rows = []
    lam = lambda_from_gamma0()
    for n in range(0, 27):
        dn = D(n)
        if dn <= 0: break
        rows.append({
            "n": n,
            "D_n": dn,
            "ln D_n": math.log(dn),
            "phi_n": phi_n(n)
        })
    return pd.DataFrame(rows)

ladder_df = ladder_table()
ladder_df.head(10)

### Calibration free ratio tests

Three quick checks that depend only on the ladder structure:
\[\frac{\varphi_{12}}{\varphi_{10}}=\left(\frac{36}{40}\right)^\lambda,\quad
  \frac{\varphi_{15}}{\varphi_{12}}=\left(\frac{30}{36}\right)^\lambda,\quad
  \frac{\varphi_{25}}{\varphi_{15}}=\left(\frac{10}{30}\right)^\lambda.\]

In [ ]:
lam = lambda_from_gamma0()
ratios_theory = [
    (phi_n(12)/phi_n(10), (36/40)**lam),
    (phi_n(15)/phi_n(12), (30/36)**lam),
    (phi_n(25)/phi_n(15), (10/30)**lam),
]
ratios_theory

### Ladder plot

In [ ]:
plt.figure()
plt.plot(ladder_df["n"], ladder_df["phi_n"], marker="o")
plt.xlabel("structure step n")
plt.ylabel("phi_n")
plt.title("E8 ladder: phi_n versus n")
plt.show()

## Block constants and observable maps

For a block \(B\) with effective rank \(r_B\) and a rational topological number \(k_B\), define
\[\zeta_B = (\pi c_3)\,\exp\!\left[-\beta_B \pi c_3\right]\exp\!\left[-\frac{k_B}{c_3}\right],\qquad \beta_B=\frac{8-r_B}{8},\]
and map to a dimensionful quantity via \(X_B=\zeta_B\,M_{\rm Pl}\,\varphi_{n_B}\).

In [ ]:
MPl = 1.221e19  # GeV

def zeta_B(rB, kB):
    betaB = (8.0 - rB)/8.0
    return (math.pi*c3())*math.exp(-betaB*math.pi*c3())*math.exp(-kB/c3())

def EW_block_values(g2=0.652, g1_SM=0.357):
    # EW block: n=12, r=2 => beta=3/4, k=41/32
    rB, kB, nB = 2.0, (41.0/32.0), 12
    z = zeta_B(rB, kB)
    vH = z*MPl*phi_n(nB)
    MW = 0.5*g2*vH
    MZ = 0.5*math.sqrt(g2**2 + g1_SM**2)*vH
    yt_top = 1.0
    mt = vH/math.sqrt(2.0)*yt_top
    return {"vH": vH, "MW": MW, "MZ": MZ, "mt(minimal)": mt}

def hadron_block_values():
    # Proton: n=15, r=5 => beta=3/8, k_p = 3/2
    rB, k_p, nB = 5.0, 1.5, 15
    z_p = zeta_B(rB, k_p)
    mp = z_p*MPl*phi_n(nB)
    # Pion: n=16, same r=5, k_pi = 51/32
    k_pi, n_pi = (51.0/32.0), 16
    z_pi = zeta_B(rB, k_pi)
    f_pi = z_pi*MPl*phi_n(n_pi)  # in GeV
    # GMOR estimate
    mu_plus_md = 6.8e-3 # GeV at 2 GeV
    qbarq_cuberoot = 0.272 # GeV
    qbarq = qbarq_cuberoot**3
    m_pi = math.sqrt(mu_plus_md*qbarq)/f_pi
    return {"mp": mp, "f_pi": f_pi*1e3, "m_pi": m_pi*1e3}  # f_pi, m_pi in MeV

def pq_axion_block():
    # PQ: n=10, r=1 => beta=7/8, k=1/2
    rB, kB, nB = 1.0, 0.5, 10
    z = zeta_B(rB, kB)
    f_a = z*MPl*phi_n(nB)      # GeV
    m_a_eV = 5.7e-6 * (1.0e12/f_a)  # eV (5.7 micro eV * (1e12 GeV / f_a))
    return {"f_a": f_a, "m_a [micro eV]": m_a_eV*1e6}

def seesaw_block(y_nu3=1.0):
    # Seesaw: n=5, r=4 => beta=1/2, k=1/8
    rB, kB, nB = 4.0, 1.0/8.0, 5
    z = zeta_B(rB, kB)
    MR = z*MPl*phi_n(nB)   # GeV
    vH = EW_block_values()["vH"]
    mnu3 = (vH**2)/MR * (y_nu3**2)   # GeV
    return {"M_R": MR, "m_nu3 [eV]": mnu3*1e9}

EW_block_values(), hadron_block_values(), pq_axion_block(), seesaw_block()

### Summary of block outputs

In [ ]:
summary = {}
summary.update({k:f for k,f in EW_block_values().items()})
for k,v in hadron_block_values().items():
    summary[k] = v
summary.update(pq_axion_block())
summary.update(seesaw_block())

pd.DataFrame([summary]).T.rename(columns={0:"value"})

### Cubic function \(f(\alpha)\) near the root

In [ ]:
def f_alpha(a):
    c = c3()
    b1 = 41.0/10.0
    return a**3 - 2*(c**3)*a**2 - 8*b1*(c**6)*math.log(1.0/phi0())

xs = np.linspace(alpha_practice*0.98, alpha_practice*1.02, 200)
ys = [f_alpha(x) for x in xs]
plt.figure()
plt.plot(xs, ys)
plt.axhline(0, linewidth=1)
plt.axvline(alpha_exact, linestyle="--", linewidth=1)
plt.xlabel("alpha")
plt.ylabel("f(alpha)")
plt.title("Cubic fixed point: root marker at alpha_exact")
plt.show()

## Inflation module (alpha attractor)

We use the TFPT normalisation \(\alpha_{\rm inf}=\varphi_0/(2c_3)\).
Predictions at the CMB pivot:
\[\;n_s \simeq 1-\frac{2}{N},\quad r\simeq\frac{12\,\alpha_{\rm inf}}{N^2},\quad
\alpha_s \simeq -\frac{2}{N^2},\quad n_t\simeq -\frac{r}{8}.\]
Amplitude \(A_s \approx 2.11\times10^{-9}\) fixes \(H\) and \(V^{1/4}\).

In [ ]:
def alpha_inf():
    return phi0()/(2.0*c3())  # unique choice per v1.06

def inflation_numbers(N, As=2.11e-9):
    ainf = alpha_inf()
    ns = 1.0 - 2.0/N
    r = 12.0*ainf/(N**2)
    alpha_s = -2.0/(N**2)
    n_t = -r/8.0
    # H and V^{1/4}
    H = math.pi * 1.221e19 * math.sqrt(As*r)/math.sqrt(2.0)  # GeV
    V_quart = (3*(math.pi**2)*As*r)**0.25 * 1.221e19  # GeV
    return {"N":N, "ns":ns, "r":r, "alpha_s":alpha_s, "n_t":n_t, "H [GeV]":H, "V^{1/4} [GeV]":V_quart}

# Example at N=55
inflation_numbers(55)

### Plots: \(n_s(N)\) and \(r(N)\)

In [ ]:
Ns = np.arange(45, 71)
vals_ns = [inflation_numbers(int(N))["ns"] for N in Ns]
vals_r  = [inflation_numbers(int(N))["r"]  for N in Ns]

plt.figure()
plt.plot(Ns, vals_ns, marker="o")
plt.xlabel("N")
plt.ylabel("n_s")
plt.title("n_s versus N")
plt.show()

plt.figure()
plt.plot(Ns, vals_r, marker="o")
plt.xlabel("N")
plt.ylabel("r")
plt.title("r versus N")
plt.show()

## Interactive playground

Move the sliders to explore block outputs and inflation numbers. Requires ipywidgets in your environment.


In [ ]:
try:
    import ipywidgets as W
    from IPython.display import display, HTML
    
    # Inflation widget
    N_slider = W.IntSlider(value=55, min=45, max=70, step=1, description="N")
    out_inf = W.Output()
    def _update_inf(change=None):
        with out_inf:
            out_inf.clear_output()
            res = inflation_numbers(N_slider.value)
            display(pd.DataFrame([res]))
    N_slider.observe(_update_inf, names="value")
    _update_inf()
    display(W.VBox([W.HTML("<b>Inflation numbers</b>"), N_slider, out_inf]))
    
    # Block widget
    rB = W.FloatSlider(value=2.0, min=1.0, max=7.0, step=0.5, description="r_B")
    kB = W.FloatText(value=41.0/32.0, description="k_B")
    nB = W.IntSlider(value=12, min=1, max=26, step=1, description="n_B")
    out_block = W.Output()
    def _update_block(change=None):
        with out_block:
            out_block.clear_output()
            z = zeta_B(rB.value, kB.value)
            X = z*MPl*phi_n(nB.value)
            display(pd.DataFrame([{"zeta_B":z, "X_B = zeta_B MPl phi_n": X,
                                   "r_B": rB.value, "k_B": kB.value, "n_B": nB.value}]))
    for w in (rB,kB,nB):
        w.observe(_update_block, names="value")
    _update_block()
    display(W.VBox([W.HTML("<b>Generic block mapper</b>"), W.HBox([rB, kB, nB]), out_block]))
except Exception as e:
    print("ipywidgets not available, skipping interactive cells:", e)

## Final readout

A compact readout of the core numbers, useful for quick checks.

In [ ]:
out = dict(
    c3=c3(),
    phi0=phi0(),
    A=A_coeff(),
    kappa=kappa(),
    alpha_exact=alpha_exact,
    alpha_inv_exact=1.0/alpha_exact,
    gamma0=gamma0(),
    lambda_val=lambda_from_gamma0(),
    alpha_inf=alpha_inf()
)
pd.DataFrame([out]).T.rename(columns={0:"value"})